<a href="https://colab.research.google.com/github/UDICatNCHU/SparkTutorial/blob/master/(Spark%20Tutorial)%20%E4%B8%8A%E8%AA%B2%E8%AC%9B%E7%BE%A9%202019_Spark_DataFrame_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
! wget -O init_env.sh https://www.dropbox.com/s/cce2nnozquh8twz/init_env%20spark%203.1.1.sh?dl=0 && \
bash init_env.sh

--2021-11-25 04:05:12--  https://www.dropbox.com/s/cce2nnozquh8twz/init_env%20spark%203.0.1.sh?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/cce2nnozquh8twz/init_env%20spark%203.0.1.sh [following]
--2021-11-25 04:05:12--  https://www.dropbox.com/s/raw/cce2nnozquh8twz/init_env%20spark%203.0.1.sh
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc27078e7ac6ce3df96a4743a089.dl.dropboxusercontent.com/cd/0/inline/BakoF12MvG-aBTDpJaTfJkPAn_u4KIg7jycD5Cgvni9PdH9K-_AJ7dqmh4mIRX6fTW-OTwuUmSdjpyaoFurOsFWC2s3MMNaqPmkP8B7zHr5BdsNu7-EeayQPFPo1TQnzOrTQW93eZxEnI-GbzuiT51iI/file# [following]
--2021-11-25 04:05:13--  https://uc27078e7ac6ce3df96a4743a089.dl.dropboxusercontent.com/cd/0/inline/BakoF12MvG-aBTDpJaTfJkPAn_u4KIg7jycD5Cgv

In [26]:
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.9-src.zip")

In [27]:
from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext()

ValueError: ignored

In [28]:
!wget -O pm25.csv "https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0"

--2021-11-25 04:05:35--  https://www.dropbox.com/s/zkn3ba7pitv83el/pm2.5Taiwan.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zkn3ba7pitv83el/pm2.5Taiwan.csv [following]
--2021-11-25 04:05:35--  https://www.dropbox.com/s/raw/zkn3ba7pitv83el/pm2.5Taiwan.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf1de17698f7673545434833114.dl.dropboxusercontent.com/cd/0/inline/Bak4KV5e_FfchNV1sqXLSEBm34olnzi4yn4gTNDagHFOnV7nplHcd2l7OY-HKL0ldVVoFrZ2KpJd74WM55G_UjkeufsCZhfop63jFIVYysRCBiNq3gUtyRogBOOrUSXqa8vtiH7DPryiwSbNrkP_NWKn/file# [following]
--2021-11-25 04:05:35--  https://ucf1de17698f7673545434833114.dl.dropboxusercontent.com/cd/0/inline/Bak4KV5e_FfchNV1sqXLSEBm34olnzi4yn4gTNDagHFOnV7nplHcd2l7OY-HKL0ldVVoFrZ2KpJd

In [ ]:
weather = sc.textFile("./pm25.csv")

In [ ]:
weather_data_rdd = weather.map(lambda line : line.split(","))
pm25schema = weather_data_rdd.first()
# print(pm25schema)

# 回想如何使用RDD計算求取2015年，大里每小時的平均pm25數值。

In [ ]:
clean_weather_data = weather_data_rdd\
                    .filter(lambda x: x!=pm25schema)\
                    .filter(remove_row_with_noise)

In [ ]:
dalipm25 = clean_weather_data.filter(lambda x: x[1] == '大里' and x[2]== "PM2.5")

In [ ]:
import math
def remove_row_with_noise (x):
    for i in range(3, len(x)):
        if not x[i].isdecimal():
            return False
    return True 

def hourKeyGen(x):
    hourkeypair = []
    x=x[3:]
    for i, value in enumerate(x):
      print(i, value)
      hourkeypair.append((i, float(value)))
    return hourkeypair

count = dalipm25.count()
HourSum = dalipm25\
            .flatMap(hourKeyGen)\
            .reduceByKey(lambda x,y: x+y)\
            .mapValues(lambda x: x/count)\
            .map(lambda x: (x[1],x[0])).top(24)

print(HourSum)

[(35.43939393939394, 11), (35.38383838383838, 12), (34.92424242424242, 10), (34.59090909090909, 13), (33.43434343434343, 14), (32.80808080808081, 9), (32.63636363636363, 21), (32.505050505050505, 20), (32.474747474747474, 15), (32.18181818181818, 22), (31.747474747474747, 19), (31.166666666666668, 16), (30.939393939393938, 23), (30.818181818181817, 17), (30.63131313131313, 18), (30.04040404040404, 0), (29.636363636363637, 8), (29.1010101010101, 1), (28.1010101010101, 2), (27.065656565656564, 3), (26.51010101010101, 7), (25.853535353535353, 4), (25.055555555555557, 5), (24.883838383838384, 6)]


# 使用DataFrame 來計算每小時平均值

In [ ]:
print(dalipm25.first())

['2015/01/01', '大里', 'PM2.5', '53', '55', '58', '53', '43', '36', '35', '42', '55', '64', '65', '59', '52', '44', '47', '41', '43', '40', '42', '35', '28', '20', '18', '16']


In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql import Row

dalipm25row = dalipm25.map(lambda p:
        Row(
        date = p[0],
        location = p[1],
        measure = p[2],
        hr_01 = float(p[3]), hr_02 = float(p[4]),hr_03 = float(p[5]),hr_04 = float(p[6]),hr_05 = float(p[7]),
        hr_06 = float(p[8]),hr_07 = float(p[9]),hr_08 = float(p[10]),hr_09 = float(p[11]),hr_10 = float(p[12]),
        hr_11 = float(p[13]),hr_12 = float(p[14]),hr_13 = float(p[15]),hr_14 = float(p[16]),hr_15 = float(p[17]),
        hr_16 = float(p[18]),hr_17 = float(p[19]),hr_18 = float(p[20]),hr_19 = float(p[21]),hr_20 = float(p[22]),
        hr_21 = float(p[23]),hr_22 = float(p[24]),hr_23 = float(p[25]),hr_24 = float(p[26]),
    )
)

dalipm25row.take(5)

[Row(date='2015/01/01', hr_01=53.0, hr_02=55.0, hr_03=58.0, hr_04=53.0, hr_05=43.0, hr_06=36.0, hr_07=35.0, hr_08=42.0, hr_09=55.0, hr_10=64.0, hr_11=65.0, hr_12=59.0, hr_13=52.0, hr_14=44.0, hr_15=47.0, hr_16=41.0, hr_17=43.0, hr_18=40.0, hr_19=42.0, hr_20=35.0, hr_21=28.0, hr_22=20.0, hr_23=18.0, hr_24=16.0, location='大里', measure='PM2.5'),
 Row(date='2015/01/02', hr_01=21.0, hr_02=22.0, hr_03=26.0, hr_04=23.0, hr_05=20.0, hr_06=18.0, hr_07=15.0, hr_08=21.0, hr_09=21.0, hr_10=25.0, hr_11=29.0, hr_12=32.0, hr_13=34.0, hr_14=29.0, hr_15=32.0, hr_16=39.0, hr_17=51.0, hr_18=51.0, hr_19=47.0, hr_20=43.0, hr_21=43.0, hr_22=48.0, hr_23=47.0, hr_24=53.0, location='大里', measure='PM2.5'),
 Row(date='2015/01/03', hr_01=48.0, hr_02=48.0, hr_03=43.0, hr_04=38.0, hr_05=37.0, hr_06=36.0, hr_07=37.0, hr_08=34.0, hr_09=37.0, hr_10=46.0, hr_11=64.0, hr_12=77.0, hr_13=83.0, hr_14=75.0, hr_15=68.0, hr_16=69.0, hr_17=64.0, hr_18=65.0, hr_19=59.0, hr_20=66.0, hr_21=71.0, hr_22=66.0, hr_23=57.0, hr_24=48.0

In [ ]:
df = spark.createDataFrame(dalipm25row)
df.show()

+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|      date|hr_01|hr_02|hr_03|hr_04|hr_05|hr_06|hr_07|hr_08|hr_09|hr_10|hr_11|hr_12|hr_13|hr_14|hr_15|hr_16|hr_17|hr_18|hr_19|hr_20|hr_21|hr_22|hr_23|hr_24|location|measure|
+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|2015/01/01| 53.0| 55.0| 58.0| 53.0| 43.0| 36.0| 35.0| 42.0| 55.0| 64.0| 65.0| 59.0| 52.0| 44.0| 47.0| 41.0| 43.0| 40.0| 42.0| 35.0| 28.0| 20.0| 18.0| 16.0|      大里|  PM2.5|
|2015/01/02| 21.0| 22.0| 26.0| 23.0| 20.0| 18.0| 15.0| 21.0| 21.0| 25.0| 29.0| 32.0| 34.0| 29.0| 32.0| 39.0| 51.0| 51.0| 47.0| 43.0| 43.0| 48.0| 47.0| 53.0|      大里|  PM2.5|
|2015/01/03| 48.0| 48.0| 43.0| 38.0| 37.0| 36.0| 37.0| 34.0| 37.0| 46.0| 64.0| 77.0| 83.0| 75.0| 68.0| 69.0| 64.0| 65.0| 59.0| 66.

# 直接透過rdd生成data frame. toDF()

In [ ]:
dalipm25.toDF().show()

+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|        _1| _2|   _3| _4| _5| _6| _7| _8| _9|_10|_11|_12|_13|_14|_15|_16|_17|_18|_19|_20|_21|_22|_23|_24|_25|_26|_27|
+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|2015/01/01| 大里|PM2.5| 53| 55| 58| 53| 43| 36| 35| 42| 55| 64| 65| 59| 52| 44| 47| 41| 43| 40| 42| 35| 28| 20| 18| 16|
|2015/01/02| 大里|PM2.5| 21| 22| 26| 23| 20| 18| 15| 21| 21| 25| 29| 32| 34| 29| 32| 39| 51| 51| 47| 43| 43| 48| 47| 53|
|2015/01/03| 大里|PM2.5| 48| 48| 43| 38| 37| 36| 37| 34| 37| 46| 64| 77| 83| 75| 68| 69| 64| 65| 59| 66| 71| 66| 57| 48|
|2015/01/04| 大里|PM2.5| 60| 56| 53| 43| 53| 53| 52| 44| 44| 50| 49| 51| 45| 42| 40| 38| 36| 43| 51| 63| 68| 72| 66| 58|
|2015/01/05| 大里|PM2.5| 48| 42| 42| 34| 34| 28| 34| 35| 45| 47| 54| 46| 35| 19| 16| 21| 24| 28| 37| 52| 60| 62| 64| 61|
|2015/01/06| 大里|PM2.5| 59| 40| 34| 25| 27| 29| 2

In [ ]:
pm25schema = weather_data_rdd.first()
dalipm25.toDF(pm25schema).show()

+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|        日期| 測站|   測項| 00| 01| 02| 03| 04| 05| 06| 07| 08| 09| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23|
+----------+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|2015/01/01| 大里|PM2.5| 53| 55| 58| 53| 43| 36| 35| 42| 55| 64| 65| 59| 52| 44| 47| 41| 43| 40| 42| 35| 28| 20| 18| 16|
|2015/01/02| 大里|PM2.5| 21| 22| 26| 23| 20| 18| 15| 21| 21| 25| 29| 32| 34| 29| 32| 39| 51| 51| 47| 43| 43| 48| 47| 53|
|2015/01/03| 大里|PM2.5| 48| 48| 43| 38| 37| 36| 37| 34| 37| 46| 64| 77| 83| 75| 68| 69| 64| 65| 59| 66| 71| 66| 57| 48|
|2015/01/04| 大里|PM2.5| 60| 56| 53| 43| 53| 53| 52| 44| 44| 50| 49| 51| 45| 42| 40| 38| 36| 43| 51| 63| 68| 72| 66| 58|
|2015/01/05| 大里|PM2.5| 48| 42| 42| 34| 34| 28| 34| 35| 45| 47| 54| 46| 35| 19| 16| 21| 24| 28| 37| 52| 60| 62| 64| 61|
|2015/01/06| 大里|PM2.5| 59| 40| 34| 25| 27| 29| 2

In [ ]:
dfpm25 = dalipm25.toDF(pm25schema)

In [ ]:
for i in dfpm25.columns[3:]:
  dfpm25 = dfpm25.withColumn(i, dfpm25[i].cast("double"))

In [ ]:
dfpm25.printSchema()

root
 |-- 日期: string (nullable = true)
 |-- 測站: string (nullable = true)
 |-- 測項: string (nullable = true)
 |-- 00: double (nullable = true)
 |-- 01: double (nullable = true)
 |-- 02: double (nullable = true)
 |-- 03: double (nullable = true)
 |-- 04: double (nullable = true)
 |-- 05: double (nullable = true)
 |-- 06: double (nullable = true)
 |-- 07: double (nullable = true)
 |-- 08: double (nullable = true)
 |-- 09: double (nullable = true)
 |-- 10: double (nullable = true)
 |-- 11: double (nullable = true)
 |-- 12: double (nullable = true)
 |-- 13: double (nullable = true)
 |-- 14: double (nullable = true)
 |-- 15: double (nullable = true)
 |-- 16: double (nullable = true)
 |-- 17: double (nullable = true)
 |-- 18: double (nullable = true)
 |-- 19: double (nullable = true)
 |-- 20: double (nullable = true)
 |-- 21: double (nullable = true)
 |-- 22: double (nullable = true)
 |-- 23: double (nullable = true)



# 使用 DataFrame.filter() 來進行row資料之條件計算 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
df.filter(df.hr_01>30).show()

+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|      date|hr_01|hr_02|hr_03|hr_04|hr_05|hr_06|hr_07|hr_08|hr_09|hr_10|hr_11|hr_12|hr_13|hr_14|hr_15|hr_16|hr_17|hr_18|hr_19|hr_20|hr_21|hr_22|hr_23|hr_24|location|measure|
+----------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-------+
|2015/01/01| 53.0| 55.0| 58.0| 53.0| 43.0| 36.0| 35.0| 42.0| 55.0| 64.0| 65.0| 59.0| 52.0| 44.0| 47.0| 41.0| 43.0| 40.0| 42.0| 35.0| 28.0| 20.0| 18.0| 16.0|      大里|  PM2.5|
|2015/01/03| 48.0| 48.0| 43.0| 38.0| 37.0| 36.0| 37.0| 34.0| 37.0| 46.0| 64.0| 77.0| 83.0| 75.0| 68.0| 69.0| 64.0| 65.0| 59.0| 66.0| 71.0| 66.0| 57.0| 48.0|      大里|  PM2.5|
|2015/01/04| 60.0| 56.0| 53.0| 43.0| 53.0| 53.0| 52.0| 44.0| 44.0| 50.0| 49.0| 51.0| 45.0| 42.0| 40.0| 38.0| 36.0| 43.0| 51.0| 63.

# 使用 DataFrame.select() 來進行資料之Projection http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
df.filter(df.hr_01>50).select("hr_01","location","measure").show()

+-----+--------+-------+
|hr_01|location|measure|
+-----+--------+-------+
| 53.0|      大里|  PM2.5|
| 60.0|      大里|  PM2.5|
| 59.0|      大里|  PM2.5|
| 77.0|      大里|  PM2.5|
| 62.0|      大里|  PM2.5|
| 61.0|      大里|  PM2.5|
| 63.0|      大里|  PM2.5|
| 71.0|      大里|  PM2.5|
| 56.0|      大里|  PM2.5|
| 51.0|      大里|  PM2.5|
| 54.0|      大里|  PM2.5|
| 66.0|      大里|  PM2.5|
| 67.0|      大里|  PM2.5|
| 58.0|      大里|  PM2.5|
| 52.0|      大里|  PM2.5|
| 70.0|      大里|  PM2.5|
| 83.0|      大里|  PM2.5|
| 52.0|      大里|  PM2.5|
| 51.0|      大里|  PM2.5|
| 69.0|      大里|  PM2.5|
+-----+--------+-------+
only showing top 20 rows



## 使用 DataFrame.describe() 來進行DataFrame or Column 資料之統計 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
df.describe().show()

+-------+----------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+--------+-------+
|summary|      date|            hr_01|            hr_02|            hr_03|             hr_04|             hr_05|             hr_06|             hr_07|            hr_08|             hr_09|             hr_10|            hr_11|             hr_12|             hr_13|             hr_14|             hr_15|             hr_16|             hr_17|             hr_18|             hr_19|             hr_20|             hr_21|             hr_22|            hr_23|             hr_24|location|measure|
+-------+----------+----

# 使用 DataFrame.agg() 來進行column數值之統計計算 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
from pyspark.sql import functions as F
df.agg(F.mean(df.hr_01),F.mean(df.hr_02)).show()

+-----------------+----------------+
|       avg(hr_01)|      avg(hr_02)|
+-----------------+----------------+
|30.04040404040404|29.1010101010101|
+-----------------+----------------+



#使用 DataFrame.withColumn() 來進行column新增column http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [ ]:
df.withColumn('hr_01', df.hr_01+df.hr_02)

DataFrame[date: string, hr_01: double, hr_02: double, hr_03: double, hr_04: double, hr_05: double, hr_06: double, hr_07: double, hr_08: double, hr_09: double, hr_10: double, hr_11: double, hr_12: double, hr_13: double, hr_14: double, hr_15: double, hr_16: double, hr_17: double, hr_18: double, hr_19: double, hr_20: double, hr_21: double, hr_22: double, hr_23: double, hr_24: double, location: string, measure: string]

# 練習1: 請算算看2015全國哪個測站，紫爆天數最多？

# 練習2: 請使用SPARK SQL求取2015年，全國pm2.5最高的前十個工作站測點以及其日期。

# 使用 Spark SQL來下達SQL查詢

In [ ]:
df.registerTempTable("DaliTable")

In [ ]:
spark.sql("""
            select * from DaliTable where date ='2015/02/07'
          """).show()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 10))



AnalysisException: ignored

In [ ]:
spark.sql("""
                select count(*) count 
                from DaliTable 
                where hr_01 > 100
               """).show()

In [ ]:
spark.sql("""
                select AVG(hr_01) count 
                from DaliTable
               """).show()

In [ ]:
spark.sql("""
                select date,location, hr_01, hr_02, hr_01-hr_02 as diff 
                from DaliTable
          """).show()

In [ ]:
spark.sql("""
                select date,location, hr_01, hr_02, hr_01+hr_02 as diff 
                from DaliTable 
                order by diff DESC
                """).show()

In [ ]:
spark.sql("""
                select date,location, hr_01+hr_02+hr_03+hr_04+hr_05+hr_06+hr_07+hr_08 as sum 
                from DaliTable 
                order by sum DESC
                """).show()

# 利用DataFrame 直接讀取CSV, Fillna, DropNa功能清洗資料

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

from pyspark.sql import SQLContext
from pyspark.sql import Row

In [ ]:
df = spark.read.load("./pm25.csv", format="csv")

In [ ]:
df.show()

+----------+---+----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|       _c0|_c1|       _c2| _c3| _c4| _c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|
+----------+---+----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|        日期| 測站|        測項|  00|  01|  02|  03|  04|  05|  06|  07|  08|  09|  10|  11|  12|  13|  14|  15|  16|  17|  18|  19|  20|  21|  22|  23|
|2015/01/01| 龍潭|  AMB_TEMP|  14|  14|  14|  13|  13|  13|  12|  12|  13|  14|  14|  14|  14|  14|  13|  13|  12|  11|  11|  11|  11|  11|  11|  11|
|2015/01/01| 龍潭|        CO|0.69|0.72|0.69|0.64|0.54|0.47|0.45|0.48|0.51|0.54|0.54| 0.5|0.47|0.38|0.36|0.35|0.34|0.37|0.34|0.29|0.26|0.22|0.19|0.18|
|2015/01/01| 龍潭|        NO| 0.3| 0.1| 0.6|   2|   2| 1.9| 2.2| 3.1| 3.7| 4.3| 4.3| 4.5| 3.3| 4.1| 3.1| 3.6| 3.6|

In [ ]:
df2=df.filter(df["_c0"]!="日期") 

In [ ]:
df2.count()

465317

In [ ]:
df2.dropna()

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string]

In [ ]:
df2.count()

465317

In [ ]:
df2.filter(df2._c3.isNotNull()).show()

+----------+---+----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|       _c0|_c1|       _c2| _c3| _c4| _c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|
+----------+---+----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2015/01/01| 龍潭|  AMB_TEMP|  14|  14|  14|  13|  13|  13|  12|  12|  13|  14|  14|  14|  14|  14|  13|  13|  12|  11|  11|  11|  11|  11|  11|  11|
|2015/01/01| 龍潭|        CO|0.69|0.72|0.69|0.64|0.54|0.47|0.45|0.48|0.51|0.54|0.54| 0.5|0.47|0.38|0.36|0.35|0.34|0.37|0.34|0.29|0.26|0.22|0.19|0.18|
|2015/01/01| 龍潭|        NO| 0.3| 0.1| 0.6|   2|   2| 1.9| 2.2| 3.1| 3.7| 4.3| 4.3| 4.5| 3.3| 4.1| 3.1| 3.6| 3.6| 2.8| 2.8| 2.5| 2.2| 1.4| 2.1|   2|
|2015/01/01| 龍潭|       NO2|  11| 9.6| 8.7| 9.1| 9.6| 9.9|  11|  13|  11|  12|  12|  11|  11| 9.9| 9.9|  10|  11|

In [ ]:
for i in df2.columns[3:]:
  df2 = df2.withColumn(i, df2[i].cast("double"))

In [ ]:
df2.dropna().count()

366789

In [ ]:
df2 = df2.filter(df2["_c2"]!="RAINFALL")

# Perform Pearson Correlation using DataFrame.corr( )
# 練習: 計算大里區pm10, pm2.5 間之關聯度 http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
使用前面所建立的clean_weather_data rdd資料
    
    
    corr(col1, col2, method=None)
    Calculates the correlation of two columns of a DataFrame as a double value. Currently only supports the Pearson Correlation Coefficient. DataFrame.corr() and DataFrameStatFunctions.corr() are aliases of each other.

    Parameters:	
    col1 – The name of the first column
    col2 – The name of the second column
    method – The correlation method. Currently only supports “pearson”
    New in version 1.4.

In [ ]:
def Generated_Measurement(x):
    date = x[0]
    location = x[1]
    measure = x[2]
    measurements_of_a_day = []
    for i, value in enumerate(x[3:]):
        measurements_of_a_day.append((date, measure, "hr_"+str(i), value))
    return measurements_of_a_day

daliData = clean_weather_data.filter(lambda x: x[1]=="大里" and (x[2] == "PM2.5" or x[2] == "PM10" ))
daliData.cache()

In [ ]:
daliData.toDF().show()

In [ ]:
from pyspark.sql import Row

daliDataRow = clean_weather_data.filter(lambda x: x[1]=="大里" and (x[2] == "PM2.5" or x[2] == "PM10" ))\
    .flatMap(Generated_Measurement)\
    .map(lambda x: ( (x[0], x[2]), x[1], x[3] ) )\
    .groupBy(lambda x: x[0])\
    .filter(lambda x: len(x[1])==2)\
    .mapValues(lambda x: list(x))\
    .mapValues(lambda x: [x[0][1], x[0][2], x[1][1], x[1][2]])\
    .map(lambda x:[ x[0][0], x[0][1], x[1][1], x[1][3]])\
    .map(lambda x: Row(
            date = x[0],
            time = x[1],
            pm10 = float(x[2]),
            pm25 = float(x[3])
        ))
    
df = spark.createDataFrame(daliDataRow)

In [ ]:
df.show(10)

In [ ]:
df.groupBy().avg().show()

In [ ]:
df.corr("pm10","pm25")

In [ ]:
df.describe().show()

KeyboardInterrupt: ignored